In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
#크롤된 뉴스 데이터 불러오기
columns = ['media','date','title','article_original','url']
news_crawled_df = pd.read_csv('./crawled_news/news_df_210205_v02.csv')
news_crawled_df

,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
866,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...
867,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
868,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...
869,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [3]:
news_crawled_df.isna().sum()

media                0
date                 0
title                0
article_original    13
url                  0
dtype: int64

In [4]:
news_crawled_df.dropna(inplace=True)
news_crawled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 858 entries, 0 to 870
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   media             858 non-null    object
 1   date              858 non-null    object
 2   title             858 non-null    object
 3   article_original  858 non-null    object
 4   url               858 non-null    object
dtypes: object(5)
memory usage: 40.2+ KB


In [5]:
news_crawled_df.reset_index(inplace=True)
news_crawled_df.drop('index',inplace=True,axis=1)
news_crawled_df

,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
853,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...
854,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
855,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...
856,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [6]:
#불용어 import
stopwords2 = pd.read_csv('./불용어리스트2.txt',header=None)
stopwords = list(stopwords2[0])

In [7]:
#형태소 토크나이징
from konlpy.tag import Okt
okt = Okt()

articles = news_crawled_df.article_original

articles_morphed = []
for article in articles:
    morph = okt.morphs(article)
    
    stopped = ''
    for word in morph:
        if word not in stopwords:
            stopped += word
            stopped += ' '
            
    articles_morphed.append(stopped)

In [8]:
#형태소 분해된 컬럼 추가
news_crawled_df['article_morphed'] = articles_morphed
#날짜 칼럼 추가
news_crawled_df['date_pd']=pd.to_datetime(news_crawled_df['date'])
news_crawled_df.head()

,media,date,title,article_original,url,article_morphed,date_pd
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,사회자 소개 방탄소년단 BTS 모습 드러냈다 2019년 마지막 날 미국 뉴욕 맨해튼...,2020-01-02
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...,12월 31일 현지시각 CNN 음악 장르 스스로 재창 조 하고 인상 적 인 영향력 ...,2020-01-02
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...,경자년 庚子年 새해 에도 K 팝 스타 세계 무대 공략 계속 될 전망 특히 지난해 전...,2020-01-03
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...,K 팝 대한 관심 전 세계 적 높아지면서 서울시 이른바 ‘ K 팝 명소 ’ 들을 선...,2020-01-03
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,골든글로브 시상식 한국 최초 외국어 영화상 수상한 기생충 봉준호 감독 한 매체 인터...,2020-01-06


In [9]:
#원본 date column drop
news_crawled_df.drop(columns='date',axis=1,inplace=True)

In [20]:
#기사 날짜 14일마다 그룹화
date_grouped = []
date_range = pd.date_range(start='2020-01-01', end='2020-12-31',periods=13)
L = len(date_range)
R = len(news_crawled_df)

for i, date in enumerate(date_range):
    index_group = []
    if i< L-1:
        for j in range(R):
            if date_range[i] < news_crawled_df.date_pd[j] <= date_range[i+1]:
                index_group.append(j)
        date_grouped.append(index_group)

In [21]:
date_range

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-31 10:00:00',
               '2020-03-01 20:00:00', '2020-04-01 06:00:00',
               '2020-05-01 16:00:00', '2020-06-01 02:00:00',
               '2020-07-01 12:00:00', '2020-07-31 22:00:00',
               '2020-08-31 08:00:00', '2020-09-30 18:00:00',
               '2020-10-31 04:00:00', '2020-11-30 14:00:00',
               '2020-12-31 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [22]:
# 그룹별 tfidf 변환
def tfidfy(time_period_no):
    tfidf = TfidfVectorizer(lowercase=False)
    #그룹에 있는 아티클 가져오기
    article_group = []
    for index in date_grouped[time_period_no]:
      article_group.append(news_crawled_df.article_original[index])
    #article_group = [news_crawled_df.article_original[i] for i in date_grouped[time_period_no]]
    
    art_group_tfidf = tfidf.fit_transform(article_group)
    return art_group_tfidf

In [23]:
#date 그룹 내에서 유사도 높은 기사의 index 가져오는 함수
def get_similar_art_index(group_no, art_group_tfidf, group_index):
    # 그룹 안의 한 기사와 나머지 기사간의 similarity 비교
    sim_pair = cosine_similarity(art_group_tfidf[group_index],art_group_tfidf)
    # 인덱스 내림차순으로 소트
    sorted_index = sim_pair.argsort()[:,::-1]
    # 자기자신은 빼기
    sorted_index = sorted_index[:,1:]
    #print(sorted_index)
    
    #유사도가 큰 순으로 유사도 값을 재정렬하되 자기 자신은 제외
    art_sim_value = np.sort(sim_pair.reshape(-1))[::-1]
    art_sim_value = art_sim_value[1:]
    #print(art_sim_value)
    
    #기사의 real 인덱스의 reference point 구하기
    ref = 0
    for i in range(group_no):
        ref += len(date_grouped[i])

    # 유사도가 0.3보다 높은 real 인덱스만 뽑아오기.
    index_high = []
    for idx, value in enumerate(art_sim_value):
        if value>=0.3:
            group_index = sorted_index[0][idx]
            real_index = ref + group_index
            index_high.append(real_index)
            #유사 article로 참조된 횟수 카운터
            sim_counter[real_index] += 1
            
    return index_high

In [24]:
#참조 카운터
sim_counter = np.zeros(len(news_crawled_df))

#전체 문서에 대해, 날짜 구간 당 유사한 기사의 index를 리스트에 저장한다.
sim_list=[]
for group_no in range(len(date_grouped)):
  for idx in range(len(date_grouped[group_no])):
    art_group_tfidf = tfidfy(group_no)
    sim_list.append(get_similar_art_index(group_no, art_group_tfidf, idx))

#dataframe에 컬럼 추가
news_crawled_df['sim_articles'] = sim_list
news_crawled_df['sim_count'] = sim_counter

In [ ]:
#유사한 기사가 3개 이상이면 불러오기
idx_list = []
for i,ls in enumerate(news_crawled_df['sim_articles']):
  if len(ls)>=3:
    idx_list.append(i)
#print(idx_list)
news_crawled_df.iloc[idx_list]

In [25]:
#대표 기사 뽑기 (가장 참조가 많이 된 기사)
articles_final = []
for group in date_grouped:
    indx = group[0]
    for art_indx in group:
        if news_crawled_df.sim_count[art_indx] >= news_crawled_df.sim_count[indx]:
            indx = art_indx
    articles_final.append(indx)
articles_final

[41, 123, 139, 190, 221, 271, 312, 359, 550, 636, 779, 797]

In [26]:
# 뽑은 대표 기사 갯수
len(articles_final)

12

In [27]:
from gensim.summarization import summarize
from lexrankr import LexRank

# 출력
# 1. 유사 카운트
# 2. 날짜
# 3. 기사 제목
# 4. 기사 본문
# 5. 요약본
for indx in articles_final:
    sim_count = news_crawled_df.sim_count[indx]
#     if sim_count>2:
#     lexrank = LexRank()
#     lexrank.summarize(news_crawled_df.article_original[indx])
#     summaries = lexrank.probe(3)
    print(sim_count)
    print(news_crawled_df.date_pd[indx])
    print(news_crawled_df.title[indx],'\n')
    print(news_crawled_df.article_original[indx],'\n')
#     print(summaries,'\n')

5.0
2020-01-27 00:00:00
방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전 

 방탄소년단은 이날 미국 로스앤젤레스LA 스테이플스 센터에서 열린 제62회 그래미 시상식 2부에서 래퍼 릴 나스 엑스 컨트리 가수 빌레 레이 사이러스와 함께 ‘올드 타운 로드 올스타즈Old Tow Road AllStars’ 무대를 선보였다.  릴 나스 엑스가 먼저 자신의 히트곡 ‘올드 타운 로드’를 부르기 시작했다. 회전식 무대가 움직이며 서울을 연상케하는 배경이 펼쳐지자 방탄소년단이 등장해 서울 버전으로 각색한 ‘서울 타운 로드’ 노래를 이어 부르기 시작했다. 이 노래는 방탄소년단 멤버 RM이 지난해 7월 ‘올드 타운 로드’에 참여해 만든 음원이다.  무대 배경에는 서울 도심의 빌딩과 ‘BTS’ 방탄소년단의 팬클럽 ‘아미’ 등의 글자가 찍힌 모습이 보여 미국 내 방탄소년단의 인기를 실감케했다. 무대 마지막에는 빌리 레이 사이러스 메이슨 램지 디플로 등이 방탄소년단과 함께 올드 타운 로드 후렴구를 부르며 무대를 달궜다.  이번 공연으로 방탄소년단은 2년 연속 그래미 시상식에 참석하게 됐다. 방탄소년단은 지난해 최우수 RB 앨범 부문 시상자로 그래미 시상식 무대에 올랐다. 당시 방탄소년단은 이곳에 다시 서겠다며 그래미상 도전 의지를 보였으나 올해 후보에는 오르지 못했다.  그래미상은 세계 최고 권위의 대중음악 시상식이다. 방탄소년단은 그래미와 함께 미국 3대 대중음악 시상식으로 통하는 ‘빌보드 뮤직 어워즈’와 ‘아메리칸 뮤직 어워즈’에서 수상한 바 있다.  올해 그래미 시상식에서는 영국의 10대 여성 가수 빌리 아일리시가 올해의 레코드 앨범 노래 부문과 신인상까지 4개 부문을 수상하며 주인공으로 우뚝 섰다. 빌리 아일리시는 지난해 히트곡 ‘배드 가이’로 세계적인 인기를 얻었다. 

5.0
2020-02-28 00:00:00
방탄소년단 우한 코로나에 4월 서울콘서트 취소 

 소속사 빅히트엔터테인먼트는 28일 팬 커뮤니티 위버스를 통해 오는 4월 11일 12일 

In [28]:
news_crawled_df.article_original[61]

' BTS이 팝 가수 저스틴 비버를 제치고 미국 빌보드 소셜 50 차트에서 가장 오랫동안 1위를 지킨 가수가 됐다. 빌보드는 3일현지 시각 BTS가 소셜 50 최신 차트에서 통산 164번째 1위에 오르며 비버가 2011년 3월부터 지켜온 최장163주 기록을 깼다고 보도했다.'